In [38]:
import pandas as pd
import yahoo_fin.stock_info as si
import numpy as np

etf_bond_data=['VDE', 'GDX', 'FIDU', 'XLY', 'XLP','XLV','KBE','XLK','FCOM','FUTY','IYR','AGG'] #AGG is the bond

portfolio_tg=pd.DataFrame()
for stk in etf_bond_data:
    st_data=si.get_data(stk, start_date = "11/30/2016", end_date = "11/30/2020")
    portfolio_tg[stk]=st_data['adjclose']
print(portfolio_tg.iloc[-5:],'\n')

ptf_rtn_tg=portfolio_tg.pct_change()
ptf_rtn_tg.head(10)
ptf_logrtn_tg= portfolio_tg.apply(np.log).diff()
ptf_logrtn_tg.head(10)
cov_matrix_annual_tg = ptf_logrtn_tg.cov() * 252
print('covariance matrix for the etf"s'"\n")
print(cov_matrix_annual_tg)


                  VDE        GDX       FIDU         XLY        XLP  \
2020-11-20  48.430000  35.900002  45.520000  153.690002  66.500000   
2020-11-23  51.900002  34.360001  46.290001  155.380005  66.519997   
2020-11-24  54.529999  33.419998  47.080002  157.690002  66.970001   
2020-11-25  53.349998  34.040001  46.650002  157.880005  66.930000   
2020-11-27  52.740002  34.230000  46.580002  158.220001  66.919998   

                   XLV        KBE         XLK       FCOM       FUTY  \
2020-11-20  108.800003  38.490002  120.110001  42.790001  40.930000   
2020-11-23  108.519997  39.169998  120.080002  42.959999  41.000000   
2020-11-24  108.839996  41.070000  121.750000  43.700001  41.529999   
2020-11-25  108.440002  40.470001  122.010002  43.700001  41.599998   
2020-11-27  109.440002  39.959999  122.660004  43.980000  41.180000   

                  IYR         AGG  
2020-11-20  84.919998  118.290001  
2020-11-23  84.849998  118.260002  
2020-11-24  85.089996  118.099998  
2020-11-

In [25]:
import numpy as np
import random as random

def sum_to_x(n,x):
    r = [random.uniform(-0.1,1) for i in range(1,n)]
    r.append(sum(r)*0.25)
    s=sum(r)
    r = [ i/s for i in r ]
    return r

print(sum_to_x(12,1))
vol=[]
rtn=[]


for i in range(5000):
    weights_tmp= sum_to_x(12,1)
    weights=np.array(weights_tmp)
    ptf_variance = np.dot(weights.T, np.dot(cov_matrix_annual_tg, weights))
    #print(ptf_variance)
    ptf_volatility = np.sqrt(ptf_variance)
    #print(ptf_volatility)
    ptf_rtn = np.sum(ptf_logrtn_tg.mean()*weights) * 252
    vol.append(ptf_volatility)
    rtn.append(ptf_rtn)
    
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=vol, y=rtn, mode='markers'))
[0.16154047358631396, 0.14377135362874552, 0.03579624450836528, 0.010564782549784674, 0.11754331782706934, 0.08627127288440471, -0.0012839201456532555, 0.04219873940274618, 0.022049802226144475, 0.05857040872924966, 0.12297752480282943, 0.19999999999999998]
fig.show()

[0.1259243363294075, 0.17238833477899157, -0.006893888474318478, 0.007056104824868993, 0.15631689267911436, 0.07083822002249661, 0.1876781278179383, -0.013864095451085535, 0.06894254985394274, -0.005775550731037083, 0.03738896834968092, 0.19999999999999998]


In [26]:
ptf_df=pd.DataFrame()
cnst_rtn=0.05

for i in range(25000):
    weights_tmp=sum_to_x(12, 1)
    weights=np.array(weights_tmp)
    ptf_variance = np.dot(weights.T, np.dot(cov_matrix_annual_tg, weights))
    ptf_volatility = np.sqrt(ptf_variance)
    ptf_rtn = np.sum((ptf_logrtn_tg.mean()*weights)) * 252
    sharpe_ratio=(ptf_rtn-cnst_rtn)/ptf_volatility    
    ptf_df=ptf_df.append({"vol":ptf_volatility, "rtn":ptf_rtn, "sharpe":sharpe_ratio, "wgt":weights},ignore_index=True)
    
print('Best Sharpe Ratio-')
print(ptf_df.sort_values(by=['sharpe'], ascending=False).head(20))

Best Sharpe Ratio-
            rtn    sharpe       vol  \
16909  0.161895  0.715611  0.156363   
24587  0.157870  0.682632  0.158021   
16566  0.153393  0.677495  0.152611   
7372   0.150360  0.661679  0.151674   
12683  0.153471  0.652412  0.158598   
16604  0.151341  0.630893  0.160631   
24785  0.144999  0.618752  0.153534   
14940  0.138071  0.608778  0.144668   
7827   0.138499  0.607024  0.145791   
1487   0.143599  0.601801  0.155532   
8229   0.140925  0.599461  0.151678   
12498  0.136229  0.598824  0.143997   
5949   0.136316  0.593970  0.145321   
2815   0.137786  0.593806  0.147837   
18653  0.138147  0.593162  0.148605   
13295  0.146075  0.591152  0.162522   
48     0.141525  0.589800  0.155179   
4885   0.136693  0.588715  0.147257   
20380  0.133137  0.585463  0.142002   
21294  0.138343  0.581488  0.151926   

                                                     wgt  
16909  [-0.025323702149298927, 0.09909118575444681, 0...  
24587  [-0.023355740162487565, 0.0694052324

In [27]:
max_sharpe_idx = np.argmax(ptf_df['sharpe'])
sdp, rp = ptf_df.iloc[max_sharpe_idx]['vol'], ptf_df.iloc[max_sharpe_idx]['rtn']
print(sdp, rp)
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=ptf_df['vol'], y=ptf_df['rtn'], mode='markers'))

fig.update_traces(marker=dict(size=1,
                              line=dict(width=1,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
#fig.update_xaxes(range=[0.02, 0.2])
fig.add_trace(go.Scatter(x=[sdp], y=[rp], mode='markers', marker=dict(size=5)))

fig.show()

0.1563630634665024 0.16189518539108372


In [30]:
print(ptf_df.wgt[16909])

[-2.53237021e-02  9.90911858e-02  2.24070610e-02  2.94464385e-01
  1.60243426e-05  4.65962444e-02 -2.96651080e-02  1.93310904e-01
  1.90379172e-01  4.02256729e-02 -3.15018397e-02  2.00000000e-01]
